In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
import math

seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("sample 80mm.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)


       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    1.726000e+03    1.726000e+03     1.726000e+03     1.726000e+03   
mean     7.115570e-12    1.157015e-13     1.710815e-13     5.559731e-13   
std      1.361107e-13    1.317890e-13     1.091516e-13     6.867076e-14   
min      6.429800e-12    5.700000e-15     0.000000e+00     1.940000e-14   
25%      7.039125e-12    2.280000e-14     3.072500e-14     5.518000e-13   
50%      7.118550e-12    2.980000e-14     2.416500e-13     5.661500e-13   
75%      7.186275e-12    2.974000e-13     2.569000e-13     5.805750e-13   
max      1.067000e-11    3.537000e-13     3.447000e-13     6.415000e-13   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     1.726000e+03     1.724000e+03     5.800000e+01     1.100000e+01   
mean      4.589838e-13     1.513647e-13     1.230638e-13     1.458636e-13   
std       5.295281e-14     1.069326e-13     1.014574e-13     9.972446e-14   
min       7.0200

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/15 15:30:14.401,2022/11/15 15:30:14.651,2022/11/15 15:30:14.901,2022/11/15 15:30:15.151,2022/11/15 15:30:15.401,2022/11/15 15:30:15.651,2022/11/15 15:30:15.902,2022/11/15 15:30:16.152,2022/11/15 15:30:16.402,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
3,2022/11/15 15:30:16.724,2022/11/15 15:30:16.974,2022/11/15 15:30:17.224,2022/11/15 15:30:17.475,2022/11/15 15:30:17.725,2022/11/15 15:30:17.975,2022/11/15 15:30:18.225,2022/11/15 15:30:18.726,NaN,2.323,2.323,2.323,2.324,2.324,2.324,2.323,2.574,NaN
4,2022/11/15 15:30:19.047,2022/11/15 15:30:19.297,2022/11/15 15:30:19.547,2022/11/15 15:30:19.798,2022/11/15 15:30:20.048,2022/11/15 15:30:20.298,2022/11/15 15:30:20.548,2022/11/15 15:30:21.049,NaN,4.646,4.646,4.646,4.647,4.647,4.647,4.646,4.897,NaN
5,2022/11/15 15:30:21.370,2022/11/15 15:30:21.620,2022/11/15 15:30:22.120,2022/11/15 15:30:22.370,2022/11/15 15:30:22.621,2022/11/15 15:30:23.371,NaN,NaN,NaN,6.969,6.969,7.219,7.219,7.220,7.720,NaN,NaN,NaN
6,2022/11/15 15:30:23.692,2022/11/15 15:30:23.942,2022/11/15 15:30:24.443,2022/11/15 15:30:24.693,2022/11/15 15:30:24.943,2022/11/15 15:30:25.694,NaN,NaN,NaN,9.291,9.291,9.542,9.542,9.542,10.043,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,2022/11/15 16:36:51.425,2022/11/15 16:36:51.675,2022/11/15 16:36:52.175,2022/11/15 16:36:52.425,2022/11/15 16:36:52.676,2022/11/15 16:36:53.426,NaN,NaN,NaN,3997.024,3997.024,3997.274,3997.274,3997.275,3997.775,NaN,NaN,NaN
1724,2022/11/15 16:36:53.747,2022/11/15 16:36:53.997,2022/11/15 16:36:54.498,2022/11/15 16:36:54.748,2022/11/15 16:36:54.998,2022/11/15 16:36:55.749,NaN,NaN,NaN,3999.346,3999.346,3999.597,3999.597,3999.597,4000.098,NaN,NaN,NaN
1725,2022/11/15 16:36:56.070,2022/11/15 16:36:56.320,2022/11/15 16:36:56.820,2022/11/15 16:36:57.070,2022/11/15 16:36:57.321,2022/11/15 16:36:58.071,NaN,NaN,NaN,4001.669,4001.669,4001.919,4001.919,4001.920,4002.420,NaN,NaN,NaN
1726,2022/11/15 16:36:58.392,2022/11/15 16:36:58.642,2022/11/15 16:36:59.143,2022/11/15 16:36:59.393,2022/11/15 16:36:59.643,2022/11/15 16:37:00.394,NaN,NaN,NaN,4003.991,4003.991,4004.242,4004.242,4004.242,4004.743,NaN,NaN,NaN


### Position of the EGA

 

<div>
<img src="20221114_142012.jpg" width="1000"/>
</div>

In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,15/11/2022 15:28:48,7.0,-0.063527,5.850000e-10,3.170000e-10,4.996000e-09,5.254000e-07,0.000485,239.0,-5.163,4302.676,NaN,NaN
2,NaN,15/11/2022 15:28:56,15.0,-0.063035,5.830000e-10,3.160000e-10,4.996000e-09,5.248000e-07,0.000484,239.0,-5.275,4302.789,NaN,NaN
3,NaN,15/11/2022 15:29:03,22.0,-0.063798,5.830000e-10,3.160000e-10,4.996000e-09,5.248000e-07,0.000486,239.0,-5.163,4302.901,NaN,NaN
4,NaN,15/11/2022 15:29:10,29.0,-0.064760,5.840000e-10,3.160000e-10,4.996000e-09,5.248000e-07,0.000478,239.0,-5.163,4302.901,NaN,NaN
5,NaN,15/11/2022 15:29:17,36.0,-0.062615,5.830000e-10,3.160000e-10,4.996000e-09,5.204000e-07,0.000498,239.0,-5.275,4302.789,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.0,3366.0,3.366000e+03,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3.366000e+03,3.366000e+03
mean,2018.228343,242.886631,98.063654,978.245051,977.929834,977.428176,0.0,0.0,2.000000e-02,-0.058453,0.532728,12.071718,12.089546,12.072743,-0.009342,-0.016986,-0.011663,7.947017e-08,5.327284e-07
std,1164.458191,249.417247,13.972747,144.616752,145.801451,147.317038,0.0,0.0,1.113858e-15,0.085515,0.529528,1.812004,1.819521,1.824252,0.001704,0.001228,0.000769,6.643241e-08,5.295278e-07
min,1.172000,0.040000,0.060000,0.000000,0.000000,0.000000,0.0,0.0,2.000000e-02,-0.493900,0.006400,0.009400,0.008000,0.006200,-0.014400,-0.019400,-0.013600,0.000000e+00,6.400000e-09
25%,1010.336000,0.100000,100.060000,999.960000,1000.060000,1000.060000,0.0,0.0,2.000000e-02,-0.139800,0.019200,12.308000,12.329200,12.314100,-0.010100,-0.017800,-0.012200,1.940000e-08,1.920000e-08
50%,2018.473000,0.140000,100.060000,999.980000,1000.060000,1000.060000,0.0,0.0,2.000000e-02,-0.000400,0.022600,12.444250,12.464100,12.449200,-0.009000,-0.017000,-0.011600,2.340000e-08,2.260000e-08
75%,3026.082750,499.920000,100.060000,999.980000,1000.080000,1000.080000,0.0,0.0,2.000000e-02,0.019400,1.027600,12.532525,12.552500,12.538500,-0.008100,-0.015900,-0.010900,1.398000e-07,1.027600e-06
max,4033.971000,499.920000,100.140000,999.980000,1000.080000,1000.080000,0.0,0.0,2.000000e-02,0.534500,3.303100,12.634500,12.656400,12.641500,-0.006500,-0.014400,-0.010200,5.345000e-07,3.303100e-06


### Data processing

In [ ]:
#Normalising I_em values to maximum emission 


In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="emission on"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 11, 15, 15, 31, 17)
y=pd.Timestamp(2022, 11, 15, 15, 44, 16)
#then find the time value closest to "emission on" in the MID scan file 
print("\n","Result 3:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])

#then find the time value closest to "grid repelling" in the MID scan file 
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 5:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck
print("Res 6:",mid["mass 2, corr"].describe())

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][29:364]/h2_sens
print("Result 7:",p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:29]/h2_sens
print("Result 8:",base_h2.describe())

Result 1:           Date          Time  Elapsed time  VMon1   VMon2   VMon3   VMon4  \
78  15/11/2022  15:31:13.646        95.204   0.06  100.14  999.14  893.14   
79  15/11/2022  15:31:14.843        96.401   0.12  100.14  999.30  982.62   
80  15/11/2022  15:31:16.035        97.593   0.08  100.12  999.46  999.60   
81  15/11/2022  15:31:17.197        98.755   0.08  100.12  999.54  999.60   

     VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
78  680.88    0.0    0.0  ...  1.3459   9.8469   9.7662   9.3660 -0.0144   
79  770.96    0.0    0.0  ...  1.3413  10.7313  10.6603  10.3895 -0.0144   
80  901.12    0.0    0.0  ...  1.0154  10.9129  10.8848  10.8783 -0.0144   
81  992.74    0.0    0.0  ...  1.2089  11.2947  11.2655  11.2742 -0.0144   

     IMon7   IMon8      Comment       hv_grid        em  
78 -0.0182 -0.0109  emission on  9.800000e-09  0.000001  
79 -0.0182 -0.0109  emission on  8.200000e-08  0.000001  
80 -0.0182 -0.0109  emission on  8.200000e-08  0.

### Plotting

In [13]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan, EGA in the middle on a-C @ 4K, dynamic vacuum")
plt.yscale('log')

In [12]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("HiVolta current recording, EGA in the middle on a-C @ 4K, dynamic vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 14,
        }
plt.text(150, 1.9e-6, 'Emission on', fontdict = font)
plt.text(1150, 7e-9, 'Grid repelling', fontdict = font)
plt.text(2070, 1.9e-6, 'Grid transparent', fontdict = font)
plt.text(2750, 7e-9, 'Grid repelling, EGA turned', fontdict = font)
plt.text(3570, 4.5e-8, 'Grid transparent', fontdict = font)

plt.yscale('log')


In [11]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution, EGA in the middle on a-C @ 4K, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
ax.yaxis.set_minor_formatter(LogFormatter(minor_thresholds=(2,0.5)))
plt.show()

In [10]:
#Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution, EGA in the middle on a-C @ 4K, dynamic vacuum")
plt.yscale('linear')